In [ ]:
import pandas as pd
import pyodbc
import numpy as np

url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,sep=';',decimal='.')
df.drop(columns=['STATION'],inplace=True)
# ciselne sloupce
float_columns = df.select_dtypes(include=["float"])

# zaokrouhli ciselne sloupce
df[float_columns.columns] = float_columns.apply(lambda x: x.round(2))
df['NAME'] = "'"+df['NAME']+"'"
df['DATE'] = "'"+df['DATE']+"'"
df = df.replace({np.nan:"null"})


cnxn = pyodbc.connect("DRIVER={SQL Server};"
                      "SERVER=DESKTOP-PETR\PETRDESKTOP;"
                      "DATABASE=skoleni;"

                    "Trusted_Connection=yes;")
                   # )


for i,r in df.iterrows():
    cursor = cnxn.cursor()
    values = ','.join([ str(x) for x in df.iloc[i].to_list()])

    try:
        cursor.execute(f"INSERT INTO meteodata (name,date,prcp,snwd,tavg,tmax,tmin) VALUES ({values}) ")
    except:
        cursor.rollback()
    finally:    
    
        cursor.commit()
cnxn.close()